#### Text translation

**Text Translation Setup**

- Imports required libraries for sequence-to-sequence text translation using PyTorch.
- Configures device to use GPU if available, otherwise CPU.


#### Dataset Description

We use the **English-French sentence pair dataset** from [manythings.org](http://www.manythings.org/anki/fra-eng.zip).

- Contains parallel English and French sentences for basic machine translation tasks.
- Designed for beginners to explore translation models without large compute requirements.
- Suitable for quick experimentation and understanding of sequence-to-sequence models.


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Dataset Preparation for Text Translation

- Downloads and extracts the English-French sentence pair dataset.
- Defines `normalize` function to lowercase, clean, and simplify text.
- Loads sentence pairs from `fra.txt` and applies normalization.
- Keeps the first 10,000 sentence pairs for training.


In [14]:
import os
import unicodedata
import re

!wget http://www.manythings.org/anki/fra-eng.zip
!unzip -q fra-eng.zip -y

def normalize(s):
    s = unicodedata.normalize('NFD', s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s.strip()

with open("fra.txt", encoding="utf-8") as f:
    lines = f.read().strip().split('\n')

pairs = [[normalize(s) for s in l.split('\t')[:2]] for l in lines if len(l.split('\t')) >= 2]
pairs = pairs[:10000]

--2025-06-23 06:29:20--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8143096 (7.8M) [application/zip]
Saving to: ‘fra-eng.zip.2’

fra-eng.zip.2       100%[===================>]   7.77M  41.4MB/s    in 0.2s    

2025-06-23 06:29:21 (41.4 MB/s) - ‘fra-eng.zip.2’ saved [8143096/8143096]

caution: filename not matched:  -y


#### Vocabulary Creation

- `tokenize`: Splits sentences into word tokens.
- `build_vocab`: Builds word-to-index vocabulary for source and target languages with special tokens (`<pad>`, `<sos>`, `<eos>`, `<unk>`).
- Creates source (`SRC_vocab`) and target (`TGT_vocab`) vocabularies along with index-to-word mappings.


In [24]:
from collections import Counter

def tokenize(s): return s.split()

def build_vocab(pairs, idx):
    counter = Counter()
    for pair in pairs:
        counter.update(tokenize(pair[idx]))
    vocab = {'<pad>':0, '<sos>':1, '<eos>':2, '<unk>':3}
    for word in counter:
        vocab[word] = len(vocab)
    return vocab

SRC_vocab = build_vocab(pairs, 0)
TGT_vocab = build_vocab(pairs, 1)

SRC_itos = {i:s for s,i in SRC_vocab.items()}
TGT_itos = {i:s for s,i in TGT_vocab.items()}

#### Data Processing and Batching

- `encode`: Converts a sentence to a list of token indices, using `<unk>` for unknown words.
- `tensorify`: Converts sentence pairs to source and target tensors with `<sos>` and `<eos>` tokens.
- Splits data into training and validation sets (90/10 split).
- `collate_fn`: Pads source and target sequences for batch processing.
- Creates DataLoaders for training and validation with batch size 32.


In [25]:
def encode(sentence, vocab):
    return [vocab.get(word, vocab['<unk>']) for word in tokenize(sentence)]

def tensorify(pair):
    src = torch.tensor([SRC_vocab['<sos>']] + encode(pair[0], SRC_vocab) + [SRC_vocab['<eos>']], dtype=torch.long)
    tgt = torch.tensor([TGT_vocab['<sos>']] + encode(pair[1], TGT_vocab) + [TGT_vocab['<eos>']], dtype=torch.long)
    return src, tgt

from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

train_pairs, val_pairs = train_test_split(pairs, test_size=0.1, random_state=42)
tensor_train = [tensorify(p) for p in train_pairs]
tensor_val   = [tensorify(p) for p in val_pairs]

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_pad = pad_sequence(src_batch, padding_value=SRC_vocab['<pad>'])
    tgt_pad = pad_sequence(tgt_batch, padding_value=TGT_vocab['<pad>'])
    return src_pad, tgt_pad

loader = DataLoader(tensor_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(tensor_val, batch_size=32, shuffle=False, collate_fn=collate_fn)

#### Encoder-Decoder Model

- `Encoder`:  
  - Embeds source tokens and processes them with a GRU.  
  - Returns output features and final hidden state.  

- `Decoder`:  
  - Embeds target tokens and processes them with a GRU.  
  - Uses a linear layer to predict the next token.  


In [54]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.rnn(embedded)
        return outputs, hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim):
        super().__init__()
        self.embed = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim)
        self.fc_out = nn.Linear(hid_dim, output_dim)
    def forward(self, input, hidden):
        embedded = self.embed(input.unsqueeze(0))
        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden

#### Model Training and Evaluation

- Defines embedding and hidden dimensions for encoder and decoder.
- Initializes encoder, decoder, optimizers, and loss function (`CrossEntropyLoss` with padding ignored).
- `train_one_epoch`: Trains the model using teacher forcing and computes average training loss.
- `evaluate_val_loss`: Evaluates model on validation set without gradient updates.
- Runs training for 15 epochs, printing training and validation loss after each epoch.


In [27]:
INPUT_DIM = len(SRC_vocab)
OUTPUT_DIM = len(TGT_vocab)
HID_DIM = 256
EMB_DIM = 128

encoder = Encoder(INPUT_DIM, EMB_DIM, HID_DIM).to(device)
decoder = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM).to(device)

enc_opt = optim.Adam(encoder.parameters())
dec_opt = optim.Adam(decoder.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=TGT_vocab['<pad>'])

def train_one_epoch():
    encoder.train()
    decoder.train()
    total_loss = 0
    for src, tgt in loader:
        src, tgt = src.to(device), tgt.to(device)
        enc_opt.zero_grad(); dec_opt.zero_grad()

        hidden = encoder(src)
        input_dec = tgt[0,:]
        loss = 0

        for t in range(1, tgt.shape[0]):
            pred, hidden = decoder(input_dec, hidden)
            loss += criterion(pred, tgt[t])
            input_dec = tgt[t]  # teacher forcing

        loss.backward()
        enc_opt.step(); dec_opt.step()
        total_loss += loss.item() / (tgt.shape[0] - 1)
    return total_loss / len(loader)

def evaluate_val_loss():
    encoder.eval()
    decoder.eval()
    total_loss = 0

    with torch.no_grad():
        for src, tgt in val_loader:
            src, tgt = src.to(device), tgt.to(device)
            hidden = encoder(src)

            input_dec = tgt[0, :]
            loss = 0

            for t in range(1, tgt.shape[0]):
                pred, hidden = decoder(input_dec, hidden)
                loss += criterion(pred, tgt[t])
                input_dec = tgt[t]  # teacher forcing

            total_loss += loss.item() / (tgt.shape[0] - 1)

    return total_loss / len(val_loader)

for epoch in range(1, 16):
    train_loss = train_one_epoch()
    val_loss = evaluate_val_loss()
    print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 1, Train Loss: 2.9507, Val Loss: 2.2762
Epoch 2, Train Loss: 1.9661, Val Loss: 1.9119
Epoch 3, Train Loss: 1.5639, Val Loss: 1.7394
Epoch 4, Train Loss: 1.2855, Val Loss: 1.6296
Epoch 5, Train Loss: 1.0933, Val Loss: 1.5382
Epoch 6, Train Loss: 0.9226, Val Loss: 1.4748
Epoch 7, Train Loss: 0.7918, Val Loss: 1.4445
Epoch 8, Train Loss: 0.6784, Val Loss: 1.4161
Epoch 9, Train Loss: 0.5888, Val Loss: 1.3974
Epoch 10, Train Loss: 0.5145, Val Loss: 1.3863
Epoch 11, Train Loss: 0.4458, Val Loss: 1.3980
Epoch 12, Train Loss: 0.3907, Val Loss: 1.3776
Epoch 13, Train Loss: 0.3432, Val Loss: 1.3904
Epoch 14, Train Loss: 0.3075, Val Loss: 1.3953
Epoch 15, Train Loss: 0.2804, Val Loss: 1.4015


#### Translation Function

- `translate`: Translates a given source sentence using the trained encoder-decoder model.
- Performs greedy decoding until `<eos>` token or maximum length is reached.
- Returns the generated target sentence as text.


In [28]:
def translate(sentence, max_len=20):
    encoder.eval(); decoder.eval()
    with torch.no_grad():
        src_tensor = torch.tensor([SRC_vocab['<sos>']] + encode(sentence, SRC_vocab) + [SRC_vocab['<eos>']], device=device).unsqueeze(1)
        hidden = encoder(src_tensor)

        input_dec = torch.tensor([TGT_vocab['<sos>']], device=device)
        output_sentence = []

        for _ in range(max_len):
            pred, hidden = decoder(input_dec, hidden)
            top1 = pred.argmax(1).item()
            if top1 == TGT_vocab['<eos>']:
                break
            output_sentence.append(TGT_itos[top1])
            input_dec = torch.tensor([top1], device=device)

    return ' '.join(output_sentence)

#### BLEU Score Evaluation Setup

- Installs `sacrebleu` library for evaluating translation quality using BLEU scores.


In [35]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.2 MB/s eta 0:00:00


#### BLEU Score Evaluation

- `evaluate_bleu_sacre`: Computes BLEU score using `sacrebleu` on random validation samples.
- Compares model translations to reference sentences to assess translation quality.


In [39]:
import sacrebleu

def evaluate_bleu_sacre(translate_fn, val_pairs, n_samples=100):
    refs = []
    hyps = []

    samples = random.sample(val_pairs, n_samples)

    for src_tensor, tgt_tensor in samples:
        src_sentence = ' '.join([SRC_itos[i.item()] for i in src_tensor[1:-1]])
        tgt_sentence = ' '.join([TGT_itos[i.item()] for i in tgt_tensor[1:-1]])
        pred = translate_fn(src_sentence)

        refs.append([tgt_sentence])
        hyps.append(pred)

    bleu = sacrebleu.corpus_bleu(hyps, list(zip(*refs)))
    print(f"BLEU score (sacreBLEU): {bleu.score:.2f}")


#### Run BLEU Score Evaluation

- Evaluates translation quality on 100 random validation samples using `sacrebleu`.
- Prints the overall BLEU score for the model.


In [40]:
evaluate_bleu_sacre(translate, tensor_val, n_samples=100)

BLEU score (sacreBLEU): 21.97


#### Google Drive Mounting

- Mounts Google Drive to `/content/drive` for saving or loading files in Colab.


In [42]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Create Save Directory

- Creates a folder in Google Drive to store model files.
- Uses `/content/drive/MyDrive/DL/seq2seq` as the save location.


In [43]:
save_dir = "/content/drive/MyDrive/DL/seq2seq"
os.makedirs(save_dir, exist_ok=True)

#### Save Model Checkpoint

- Saves encoder, decoder, optimizer states, vocabularies, and current epoch as a `.pt` checkpoint file in the specified Drive folder.


In [45]:
torch.save({
    'encoder_state_dict': encoder.state_dict(),
    'decoder_state_dict': decoder.state_dict(),
    'enc_opt_state_dict': enc_opt.state_dict(),
    'dec_opt_state_dict': dec_opt.state_dict(),
    'epoch': epoch,
    'SRC_vocab': SRC_vocab,
    'TGT_vocab': TGT_vocab,
}, f"{save_dir}/vanilla_seq2seq_checkpoint.pt")

### RNN with attention 

#### Encoder-Decoder with Attention (Bahdanau Attention)

**Encoder Architecture**

- Uses a single-layer **GRU (Gated Recurrent Unit)** to process source sequences.
- Converts source tokens to embeddings using `nn.Embedding`.
- Produces two outputs:
  - **Encoder Outputs**: Hidden states for all time steps (used for attention).
  - **Final Hidden State**: Passed to the decoder to initialize generation.


**Decoder with Bahdanau Attention**

- Standard **GRU-based Decoder** enhanced with **Bahdanau Additive Attention**.
- Decoder Steps:
  1. Embeds current input token.
  2. Computes attention weights based on decoder hidden state and encoder outputs.
  3. Generates context vector by attending to relevant encoder hidden states.
  4. Concatenates context, embedding, and passes to GRU for next prediction.
  5. Outputs the next token prediction using a linear layer.


#### Why Use Attention?

- **Traditional Seq2Seq Limitation**: Encoder compresses the entire input sequence into a single fixed-size vector, which can cause information loss, especially for long sentences.
- **Attention Mechanism Benefits**:
  - Allows decoder to access all encoder hidden states dynamically.
  - Focuses on different parts of the input at each decoding step.
  - Improves translation quality, especially for complex or long sentences.
  - Provides interpretability by showing which source tokens the model attends to during generation.


#### Summary

This architecture combines the strengths of RNNs for sequence modeling with attention to overcome memory bottlenecks and enhance translation accuracy.




In [ ]:
import os, re, unicodedata, random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from collections import Counter

# Normalize and tokenize

def normalize(s):
    s = unicodedata.normalize('NFD', s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s.strip()

# Download
!wget -q http://www.manythings.org/anki/fra-eng.zip
!unzip -q fra-eng.zip -y

with open("fra.txt", encoding="utf-8") as f:
    lines = f.read().strip().split('\n')
pairs = [[normalize(s) for s in l.split('\t')[:2]] for l in lines][:10000]

# Build vocab

def tokenize(s): return s.split()

def build_vocab(pairs, idx):
    counter = Counter()
    for pair in pairs:
        counter.update(tokenize(pair[idx]))
    vocab = {'<pad>':0, '<sos>':1, '<eos>':2, '<unk>':3}
    for word in counter:
        vocab[word] = len(vocab)
    return vocab

SRC_vocab = build_vocab(pairs, 0)
TGT_vocab = build_vocab(pairs, 1)
SRC_itos = {i:s for s,i in SRC_vocab.items()}
TGT_itos = {i:s for s,i in TGT_vocab.items()}

# Encode

def encode(sentence, vocab):
    return [vocab.get(w, vocab['<unk>']) for w in tokenize(sentence)]

def tensorify(pair):
    src = torch.tensor([SRC_vocab['<sos>']] + encode(pair[0], SRC_vocab) + [SRC_vocab['<eos>']], dtype=torch.long)
    tgt = torch.tensor([TGT_vocab['<sos>']] + encode(pair[1], TGT_vocab) + [TGT_vocab['<eos>']], dtype=torch.long)
    return src, tgt

from sklearn.model_selection import train_test_split
train_pairs, val_pairs = train_test_split(pairs, test_size=0.1, random_state=42)
tensor_train = [tensorify(p) for p in train_pairs]
tensor_val = [tensorify(p) for p in val_pairs]

def collate_fn(batch):
    src, tgt = zip(*batch)
    return pad_sequence(src, padding_value=SRC_vocab['<pad>']), pad_sequence(tgt, padding_value=TGT_vocab['<pad>'])

train_loader = DataLoader(tensor_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(tensor_val, batch_size=32, shuffle=False, collate_fn=collate_fn)


class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.rnn(embedded)
        return outputs, hidden

class BahdanauAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.attn = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))

    def forward(self, decoder_hidden, encoder_outputs):
        src_len = encoder_outputs.shape[0]
        batch_size = encoder_outputs.shape[1]
        decoder_hidden = decoder_hidden.repeat(src_len, 1, 1)
        energy = torch.tanh(self.attn(torch.cat((decoder_hidden, encoder_outputs), dim=2)))
        energy = energy.permute(1, 2, 0)
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        attention = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attention, dim=1)

class AttnDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim + enc_hid_dim, dec_hid_dim)
        self.fc_out = nn.Linear(emb_dim + enc_hid_dim + dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        a = self.attention(hidden, encoder_outputs).unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        context = torch.bmm(a, encoder_outputs).permute(1, 0, 2)
        rnn_input = torch.cat((embedded, context), dim=2)
        output, hidden = self.rnn(rnn_input, hidden)
        output = self.fc_out(torch.cat((output.squeeze(0), context.squeeze(0), embedded.squeeze(0)), dim=1))
        return output, hidden


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256

attn = BahdanauAttention(HID_DIM, HID_DIM)
encoder = Encoder(len(SRC_vocab), ENC_EMB_DIM, HID_DIM).to(device)
attn_decoder = AttnDecoder(len(TGT_vocab), DEC_EMB_DIM, HID_DIM, HID_DIM, attn).to(device)

enc_opt = optim.Adam(encoder.parameters(), lr=0.001)
dec_opt = optim.Adam(attn_decoder.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=TGT_vocab['<pad>'])


def train_one_epoch():
    encoder.train()
    attn_decoder.train()
    total_loss = 0
    for src, tgt in train_loader:
        src, tgt = src.to(device), tgt.to(device)
        enc_opt.zero_grad()
        dec_opt.zero_grad()
        encoder_outputs, hidden = encoder(src)
        input_dec = tgt[0, :]
        loss = 0
        for t in range(1, tgt.shape[0]):
            output, hidden = attn_decoder(input_dec, hidden, encoder_outputs)
            loss += criterion(output, tgt[t])
            input_dec = tgt[t]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(encoder.parameters(), 1)
        torch.nn.utils.clip_grad_norm_(attn_decoder.parameters(), 1)
        enc_opt.step()
        dec_opt.step()
        total_loss += loss.item() / (tgt.shape[0] - 1)
    return total_loss / len(train_loader)

def evaluate_val():
    encoder.eval()
    attn_decoder.eval()
    total_loss = 0
    with torch.no_grad():
        for src, tgt in val_loader:
            src, tgt = src.to(device), tgt.to(device)
            encoder_outputs, hidden = encoder(src)
            input_dec = tgt[0, :]
            loss = 0
            for t in range(1, tgt.shape[0]):
                output, hidden = attn_decoder(input_dec, hidden, encoder_outputs)
                loss += criterion(output, tgt[t])
                input_dec = tgt[t]
            total_loss += loss.item() / (tgt.shape[0] - 1)
    return total_loss / len(val_loader)


for epoch in range(1, 16):
    train_loss = train_one_epoch()
    val_loss = evaluate_val()
    print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


caution: filename not matched:  -y
Epoch 1, Train Loss: 2.5560, Val Loss: 1.8949
Epoch 2, Train Loss: 1.4920, Val Loss: 1.5498
Epoch 3, Train Loss: 0.9882, Val Loss: 1.3833
Epoch 4, Train Loss: 0.6956, Val Loss: 1.3136
Epoch 5, Train Loss: 0.5241, Val Loss: 1.3160
Epoch 6, Train Loss: 0.4276, Val Loss: 1.3146
Epoch 7, Train Loss: 0.3674, Val Loss: 1.3376
Epoch 8, Train Loss: 0.3263, Val Loss: 1.3591
Epoch 9, Train Loss: 0.3060, Val Loss: 1.3743
Epoch 10, Train Loss: 0.2885, Val Loss: 1.4148
Epoch 11, Train Loss: 0.2751, Val Loss: 1.3960
Epoch 12, Train Loss: 0.2743, Val Loss: 1.4070
Epoch 13, Train Loss: 0.2618, Val Loss: 1.4179
Epoch 14, Train Loss: 0.2560, Val Loss: 1.4103
Epoch 15, Train Loss: 0.2478, Val Loss: 1.4250


#### Translation with Attention

- `translate`: Generates translated output using the trained encoder-decoder with attention.
- Computes attention at each decoding step to focus on relevant source tokens.
- Uses greedy decoding until `<eos>` token or maximum length is reached.
- Returns the generated target sentence as text.


In [70]:
def translate(sentence, max_len=30):
    encoder.eval()
    attn_decoder.eval()
    with torch.no_grad():
        src_tensor = torch.tensor([SRC_vocab['<sos>']] + encode(sentence, SRC_vocab) + [SRC_vocab['<eos>']], dtype=torch.long).unsqueeze(1).to(device)
        encoder_outputs, hidden = encoder(src_tensor)
        input_token = torch.tensor([TGT_vocab['<sos>']], dtype=torch.long).to(device)
        outputs = []
        for _ in range(max_len):
            output, hidden = attn_decoder(input_token, hidden, encoder_outputs)
            pred_token = output.argmax(1).item()
            if pred_token == TGT_vocab['<eos>']:
                break
            outputs.append(TGT_itos[pred_token])
            input_token = torch.tensor([pred_token], dtype=torch.long).to(device)
    return ' '.join(outputs)

#### BLEU Score Evaluation (With Attention)

- Evaluates translation quality of the attention-based model on 100 validation samples.
- Uses `sacrebleu` to calculate BLEU score and assess translation accuracy.


In [71]:
evaluate_bleu_sacre(translate, tensor_val, n_samples=100)

BLEU score (sacreBLEU): 24.60


#### Save Attention Seq2Seq Checkpoint

- Saves encoder, decoder with attention, optimizer states, vocabularies, index mappings, and epoch to a `.pth` file for later use.


In [72]:
checkpoint = {
    'encoder_state_dict': encoder.state_dict(),
    'decoder_state_dict': attn_decoder.state_dict(),
    'encoder_optimizer_state_dict': enc_opt.state_dict(),
    'decoder_optimizer_state_dict': dec_opt.state_dict(),
    'SRC_vocab': SRC_vocab,
    'TGT_vocab': TGT_vocab,
    'SRC_itos': SRC_itos,
    'TGT_itos': TGT_itos,
    'epoch': epoch
}

torch.save(checkpoint, f"{save_dir}/attention_seq2seq_checkpoint.pth")



#### Final Results: Text Translation with and without Attention

**BLEU Score Explanation**

- **BLEU (Bilingual Evaluation Understudy)** is a standard metric to evaluate machine translation quality.
- It compares machine-generated translations with human references using n-gram overlap and brevity penalties.
- BLEU ranges from **0 to 100**; higher scores mean better translation accuracy.


**Model Performance Comparison**

| Model                          | BLEU Score | Ideal BLEU (Toy Dataset) |
|--------------------------------|------------|--------------------------|
| Vanilla Seq2Seq (No Attention) | **21.97**  | 20 - 25 (Expected range) |
| Seq2Seq with Bahdanau Attention | **24.60** | 23 - 27 (Improved with attention) |


**Observations**

Attention-based model improves translation quality by dynamically focusing on relevant input words.

Both models achieve reasonable BLEU scores for a small-scale English-French dataset (~10k pairs).

BLEU above **20** is typical for simple experiments; large real-world datasets aim for **30-40+** BLEU.


**Conclusion**

- Attention mechanism significantly enhances translation performance.
- BLEU score improvement confirms better handling of longer or complex sentences.
- Future work can include exploring different attention types, larger datasets, or transformer-based models.


**References**

1. **Seq2Seq (Sequence to Sequence Learning)**  
   Sutskever, I., Vinyals, O., & Le, Q. V. (2014).  
   *Sequence to sequence learning with neural networks.*  
   Advances in Neural Information Processing Systems (NeurIPS), 27.  
   [Read Paper](https://papers.nips.cc/paper_files/paper/2014/file/a14ac55a4f27472c5d894ec1c3c743d2-Paper.pdf)

2. **Attention Mechanism**  
   Bahdanau, D., Cho, K., & Bengio, Y. (2015).  
   *Neural machine translation by jointly learning to align and translate.*  
   International Conference on Learning Representations (ICLR).  
   [https://arxiv.org/abs/1409.0473](https://arxiv.org/abs/1409.0473)

3. **Fra-Eng Dataset (ManyThings.org)**  
   Tatoeba. (n.d.).  
   *English–French sentence pairs from the Tatoeba Project* [Data set]. ManyThings.org.  
   [http://www.manythings.org/anki/](http://www.manythings.org/anki/)


